In [13]:
import re
import numpy as np
from string import punctuation
punct = set(punctuation)
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import textdistance

In [2]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [4]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [5]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [29]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    top = get_closest_match_vec(text, X, vec, 5)
    
    similarities = Counter()
    for word in top:
        similarities[word] = metric.normalized_similarity(text, word)
    closest = similarities.most_common(1)[0][0]
    return closest

In [7]:
get_closest_hybrid_match('апофиоз', X, vec, metric=textdistance.levenshtein)

'апофеоз'

In [11]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [25]:
correct = 0
total = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        
        predicted = get_closest_hybrid_match(pair[1], X, vec, metric=textdistance.levenshtein)
        if predicted == pair[0]:
            correct += 1
        total += 1
        
print(correct/total)

0.8304035157810628


In [26]:
mistakes = []
total = 0

for i in range(len(true)):
    
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        
        total += 1

In [27]:
[(wt[0], wt[1], get_closest_hybrid_match(wt[1], X, vec, metric=textdistance.levenshtein)) for wt, _ in Counter(mistakes).most_common(20)]

[('сегодня', 'седня', 'средняя'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'ваще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'остатке'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'каток'),
 ('очень', 'оооочень', 'окончательно'),
 ('это', 'ето', 'ето'),
 ('ничего', 'ничо', 'ничто'),
 ('что-то', 'чтото', 'что'),
 ('периодически', 'переодически', 'периодически'),
 ('получается', 'получаеться', 'получаться'),
 ('насчет', 'нащет', 'нищета'),
 ('здесь', 'сдесь', 'садись'),
 ('можно', 'можна', 'монтаж'),
 ('девчонки', 'девченки', 'девчонки'),
 ('что', 'што', 'шт'),
 ('наконец-то', 'наконецто', 'наконец')]

In [18]:
Counter(mistakes).most_common(20)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9),
 (('ничего', 'ничо'), 9),
 (('что-то', 'чтото'), 7),
 (('периодически', 'переодически'), 7),
 (('получается', 'получаеться'), 6),
 (('насчет', 'нащет'), 6),
 (('здесь', 'сдесь'), 6),
 (('можно', 'можна'), 5),
 (('девчонки', 'девченки'), 5),
 (('что', 'што'), 5),
 (('наконец-то', 'наконецто'), 5)]

In [19]:
get_closest_hybrid_match('кстате', X, vec, metric=textdistance.levenshtein)

'остатке'

In [28]:
get_closest_match_vec('кстате', X, vec, 10)

['текста',
 'таскает',
 'стекает',
 'остатке',
 'текст',
 'статике',
 'сектант',
 'сетка',
 'касте',
 'истекает']

In [21]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [23]:
%%time
get_closest_match_with_metric('кстате', WORDS, textdistance.levenshtein)

CPU times: user 28.5 s, sys: 536 ms, total: 29 s
Wall time: 28.6 s


('кстати', 0.8333333333333334)